In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan

In [3]:
X, Y = ld_gan.data_proc.data_loader.load_data(1, verbose=1, resize = 64)

100%|██████████| 102/102 [00:33<00:00,  4.47it/s]

load data from '/export/home/oblum/projects/ld_gan/data/flowers_102/jpg_256'


In [4]:
import ld_gan
X, Y = ld_gan.data_proc.data_loader.load_data(1, verbose=1, resize = 64)
ld_gan.visualize.pre_compute_tsne("xf_11111_l512.py", X)
ld_gan.visualize.pre_compute_tsne("xf_11111_lr_clf_2.py", X)
ld_gan.visualize.pre_compute_tsne("xf_11111_nn50_5_new.py", X)

  0%|          | 0/20 [00:00<?, ?it/s]

loaded model 'projects/xf_11111_l512.py/model/e_0000.pth'
loaded model 'projects/xf_11111_l512.py/model/g_0000.pth'

  5%|▌         | 1/20 [01:51<35:25, 111.85s/it]


loaded model 'projects/xf_11111_l512.py/model/e_0050.pth'

 10%|█         | 2/20 [03:42<33:28, 111.57s/it]

...

 45%|████▌     | 9/20 [15:43<18:52, 102.96s/it]


loaded model 'projects/xf_11111_l512.py/model/e_0450.pth'
loaded model 'projects/xf_11111_l512.py/model/g_0450.pth'

 50%|█████     | 10/20 [17:26<17:10, 103.04s/it]


loaded model 'projects/xf_11111_l512.py/model/e_0500.pth'
loaded model 'projects/xf_11111_l512.py/model/g_0500.pth'


KeyboardInterrupt: 